In [0]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Activation
from keras.activations import softmax as Softmax
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import corpus_bleu
from keras.utils import to_categorical
from keras import backend as K
import numpy as np
import unicodedata
import re

Using TensorFlow backend.


In [0]:
from keras.models import model_from_json

In [0]:
BATCH_SIZE = 128  # Batch size for training
NUM_SAMPLES = 12500
EPOCHS = 50
OPTIMIZER = "adam"
EMBED_DIM = 300
HIDDEN_DIM = 256
DATA_PATH = 'enghin/hi_eng_gd.txt'
DATA_PATH_eng = 'enghin/dev.en'
DATA_PATH_hin = 'enghin/dev.hi'

In [0]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [0]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile, 'r')
    i = 0
    model = {}
    for line in f:
#         print(i)
        splitLine = line.split()
        word = splitLine[0]
#         print(word)
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
#         i += 1
    print("Done.", len(model), " words loaded!")
    return model

In [0]:
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between words and punctation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
#     w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sequence
    # so that the model know when to start and stop predicting
    w = "<SOS> " + w + " <EOS>"
    return w

In [0]:
eng_embedding = loadGloveModel('glove.6B/glove.6B.300d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [0]:
ger_embedding = loadGloveModel('wiki.hi.align.vec')

Loading Glove Model
Done. 157958  words loaded!


In [0]:
input_texts = []
target_texts = []
target_words = set()

# with open(DATA_PATH_eng, 'r', encoding='utf-8') as f:
#     lines = f.read().split("\n")
#     #print (lines)
#     for line in lines[:NUM_SAMPLES]:
# #         print (line.split('\t'))
#         try:
# #             input_text, target_text = line.split('\t')
#             input_text = preprocess_sentence(line)
# #             target_text = preprocess_sentence(target_text)
#             input_texts.append(input_text)
# #             target_texts.append(target_text)
# #             target_words.update(target_text.split())
#         except:
#             continue

# with open(DATA_PATH_hin, 'r', encoding='utf-8') as f:
#     lines = f.read().split("\n")
#     #print (lines)
#     for line in lines[:NUM_SAMPLES]:
# #         print (line.split('\t'))
#         try:
# #             input_text, target_text = line.split('\t')
# #             input_text = preprocess_sentence(input_text)
#             target_text = preprocess_sentence(line)
# #             input_texts.append(input_text)
#             target_texts.append(target_text)
#             target_words.update(target_text.split())
#         except:
#             continue

# target_words = sorted(list(target_words))            

# target_words = sorted(list(target_words))

with open(DATA_PATH, 'r', encoding='utf-8') as f:
    lines = f.read().split("\n")
#     print (lines)
    for line in lines[:2500]:
#         print (line.split('\t'))
        try:
            target_text, input_text = line.split('\t')
            input_text = preprocess_sentence(input_text)
#             print(input_text)
#             print(target)
            target_text = preprocess_sentence(target_text)
#             print(target_text)
            input_texts.append(input_text)
            target_texts.append(target_text)
            target_words.update(target_text.split())
        except:
            continue
            
# print(input_texts)
            
target_words = sorted(list(target_words))

In [0]:
max_input_seqlen = max([len(txt.split()) for txt in input_texts])
max_input_seqlen1 = 300
max_target_seqlen = max([len(txt.split()) for txt in target_texts])
max_target_seqlen1 = 300
target_vocab_size = len(target_words) + 1

In [0]:
# get decoder input data
decoder_data = []
for text in target_texts:
    tmp = []
    for word in text.split():
        embed = np.random.randn(EMBED_DIM)
        if word in ger_embedding:
            embed = ger_embedding[word]
        tmp.append(embed)
    tmp = pad_sequences(tmp, max_input_seqlen1, padding="post")
    decoder_data.append(tmp)
decoder_data = np.asarray(decoder_data)
# print(decoder_data.shape)
decoder_data = pad_sequences(decoder_data, max_target_seqlen, padding="post")

In [0]:
# get decoder target data
targword2idx = dict([(word, i + 1) for i, word in enumerate(target_words)])
idx2targword = dict((i, word) for word, i in targword2idx.items())
decoder_target_data = []
for text in target_texts:
    tmp = []
    for idx, word in enumerate(text.split()):
        if idx > 0:
            tmp.append(targword2idx[word])
    decoder_target_data.append(tmp)
decoder_target_data = pad_sequences(
    decoder_target_data, max_target_seqlen, padding="post")
decoder_target_data = to_categorical(decoder_target_data)

In [0]:
# get encoder data
encoder_data = []
for text in input_texts:
    tmp = []
    for word in text.split():
        embed = np.random.randn(EMBED_DIM)
        if word in eng_embedding:
            embed = eng_embedding[word]
        tmp.append(embed)
    tmp = pad_sequences(tmp, max_input_seqlen1, padding="post")
    encoder_data.append(tmp)
encoder_data = np.array(encoder_data)
encoder_data = pad_sequences(encoder_data, max_input_seqlen, padding="post")
# encoder_data = pad_sequences(encoder_data, max_input_seqlen, padding="post")

In [0]:
encoder_data.shape

(2499, 14, 300)

In [0]:
decoder_data.shape

(2499, 21, 300)

In [0]:
decoder_target_data.shape

(2499, 21, 2425)

In [0]:
# construct model
encoder_inputs = Input(shape=(max_input_seqlen, EMBED_DIM))
encoder_lstm = LSTM(HIDDEN_DIM, return_state=True, name="encoder_lstm")
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, EMBED_DIM))
decoder_lstm = LSTM(HIDDEN_DIM, return_sequences=True,
                    return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(
    decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(
    target_vocab_size, activation="softmax", name="decoder_dense")
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
# define training model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=OPTIMIZER,
              loss='categorical_crossentropy', metrics=["acc"])
print(model.summary())
filename = 'seq2seq_keras.h5'
checkpoint = ModelCheckpoint(
    filename, verbose=1, save_best_only=True, mode='min')
checkpoint = ModelCheckpoint(filename, verbose=1, mode='min')
model.fit([encoder_data, decoder_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS, callbacks=[checkpoint], validation_split=0.2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 14, 300)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 300)    0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 256), (None, 570368      input_1[0][0]                    
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 256),  570368      input_2[0][0]                    
                                                                 encoder_lstm[0][1]               
          

/home/akshat14714/anaconda3/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder_lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'encoder_lstm/while/Exit_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/50
1999/1999 [==============================] - 9s 5ms/step - loss: 2.1416 - acc: 0.6740 - val_loss: 3.0365 - val_acc: 0.5539

Epoch 00002: saving model to seq2seq_keras.h5
Epoch 3/50
1999/1999 [==============================] - 9s 5ms/step - loss: 1.9629 - acc: 0.6806 - val_loss: 3.0234 - val_acc: 0.5558

Epoch 00003: saving model to seq2seq_keras.h5
Epoch 4/50
1999/1999 [==============================] - 9s 5ms/step - loss: 1.9197 - acc: 0.6829 - val_loss: 3.0615 - val_acc: 0.5556

Epoch 00004: saving model to seq2seq_keras.h5
Epoch 5/50
1999/1999 [==============================] - 9s 4ms/step - loss: 1.8852 - acc: 0.6844 - val_loss: 3.0045 - val_acc: 0.5564

Epoch 00005: saving model to seq2seq_keras.h5
Epoch 6/50
1999/1999 [==============================] - 9s 4ms/step - loss: 1.8694 - acc: 0.6844 - val_loss: 3.0065 - val_acc: 0.5576

Epoch 00006: saving model to seq2seq_keras.h5
Epoch 7/50
1999/1999 [==============================] - 9s 4ms/step - loss: 1.8748 - acc: 0.683

1999/1999 [==============================] - 13s 6ms/step - loss: 1.0316 - acc: 0.8044 - val_loss: 3.7774 - val_acc: 0.5570

Epoch 00047: saving model to seq2seq_keras.h5
Epoch 48/50
1999/1999 [==============================] - 14s 7ms/step - loss: 1.0172 - acc: 0.8042 - val_loss: 3.6287 - val_acc: 0.5525

Epoch 00048: saving model to seq2seq_keras.h5
Epoch 49/50
1999/1999 [==============================] - 13s 7ms/step - loss: 0.9908 - acc: 0.8106 - val_loss: 3.7403 - val_acc: 0.5550

Epoch 00049: saving model to seq2seq_keras.h5
Epoch 50/50
1999/1999 [==============================] - 12s 6ms/step - loss: 0.9605 - acc: 0.8165 - val_loss: 3.6970 - val_acc: 0.5546

Epoch 00050: saving model to seq2seq_keras.h5


In [0]:
# create inference model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(HIDDEN_DIM,))
decoder_state_input_c = Input(shape=(HIDDEN_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
# decoder_outputs = (BATCH_SIZE, seqlen, HIDDEN_DIM)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
# decoder_outputs = (BATCH_SIZE, seqlen, target_token_size)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [0]:
def decode(input_seq):
    states = encoder_model.predict(input_seq)
    target_seq = np.random.randn(EMBED_DIM)
    target_seq = [[target_seq]]
    stop_condition = False
    prediction = ''
    sampled_word = " "

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states)
        sampled_token_idx = np.argmax(output_tokens[0, -1, :])
        try:
            sampled_word = idx2targword[sampled_token_idx]
            prediction += sampled_word + " "
        except:
            prediction +=  " "

        if (sampled_word == '<EOS>' or len(prediction) > max_target_seqlen):
            stop_condition = True

        if sampled_word in ger_embedding:
            target_seq = ger_embedding[sampled_word]
        else:
            target_seq = np.random.randn(EMBED_DIM)
        target_seq = [[target_seq]]
        states = [h, c]

    return prediction

In [0]:
actual, predicted = list(), list()

for index in range(402):
    input_seq = encoder_data[index]
    input_seq = np.expand_dims(input_seq, axis=0)
    actual.append(target_texts[index].split())
    prediction = decode(input_seq)
    predicted.append(prediction.split())
    print('-')
    print("Input sentence: ", input_texts[index])
    print("Translation: ", prediction)
    print("Truth value: ", target_texts[index])

print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' %
      corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' %
      corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
print('BLEU-4: %f' %
      corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

-
Input sentence:  <SOS> wow ! <EOS>
Translation:  वह ! <EOS> 
Truth value:  <SOS> वाह ! <EOS>
-
Input sentence:  <SOS> help ! <EOS>
Translation:  मत <EOS> 
Truth value:  <SOS> बचाओ ! <EOS>
-
Input sentence:  <SOS> jump . <EOS>
Translation:  वह मरा ! <EOS> 
Truth value:  <SOS> उछलो . <EOS>
-
Input sentence:  <SOS> jump . <EOS>
Translation:  <EOS> 
Truth value:  <SOS> कदो . <EOS>
-
Input sentence:  <SOS> jump . <EOS>
Translation:  म <EOS> 
Truth value:  <SOS> छलाग . <EOS>
-
Input sentence:  <SOS> hello ! <EOS>
Translation:  म ! ! <EOS> 
Truth value:  <SOS> नमसत। <EOS>
-
Input sentence:  <SOS> hello ! <EOS>
Translation:  मन ! <EOS> 
Truth value:  <SOS> नमसकार। <EOS>
-
Input sentence:  <SOS> cheers ! <EOS>
Translation:  मर <EOS> 
Truth value:  <SOS> वाह-वाह ! <EOS>
-
Input sentence:  <SOS> cheers ! <EOS>
Translation:  वह ! <EOS> 
Truth value:  <SOS> चियरस ! <EOS>
-
Input sentence:  <SOS> got it ? <EOS>
Translation:  मन कहा ह। <EOS> 
Truth value:  <SOS> समझ कि नही ? <EOS>
-
Input sentence:

-
Input sentence:  <SOS> how are you ? <EOS>
Translation:  यह कहा ह ? <EOS> 
Truth value:  <SOS> तम कस हो ? <EOS>
-
Input sentence:  <SOS> how are you ? <EOS>
Translation:  वह ह ह ? <EOS> 
Truth value:  <SOS> तम कसी हो ? <EOS>
-
Input sentence:  <SOS> how are you ? <EOS>
Translation:  मझ बहत ह ह। <EOS> 
Truth value:  <SOS> त कसा ह ? <EOS>
-
Input sentence:  <SOS> how are you ? <EOS>
Translation:  म कया ? <EOS> 
Truth value:  <SOS> त कसी ह ? <EOS>
-
Input sentence:  <SOS> how are you ? <EOS>
Translation:  वह घर ह। <EOS> 
Truth value:  <SOS> आप कसी ह ? <EOS>
-
Input sentence:  <SOS> how are you ? <EOS>
Translation:  यह कहा ह। <EOS> 
Truth value:  <SOS> आप कस हो ? <EOS>
-
Input sentence:  <SOS> i am hungry . <EOS>
Translation:  मझ यह म ह ह। <EOS> 
Truth value:  <SOS> मझ भख लगी ह। <EOS>
-
Input sentence:  <SOS> i like both . <EOS>
Translation:  हम कल ह ह। <EOS> 
Truth value:  <SOS> मझ दोनो पसद ह। <EOS>
-
Input sentence:  <SOS> i like cake . <EOS>
Translation:  वह स स स ह। <EOS> 
Truth valu

-
Input sentence:  <SOS> i like history . <EOS>
Translation:  म कहा ह। ह। <EOS> 
Truth value:  <SOS> मझ इतिहास पसद ह। <EOS>
-
Input sentence:  <SOS> i like the dog . <EOS>
Translation:  म म म नही ह। ह। <EOS> 
Truth value:  <SOS> मझ यह कतता अचछा लगता ह। <EOS>
-
Input sentence:  <SOS> i must buy one . <EOS>
Translation:  उसन की का का ह। <EOS> 
Truth value:  <SOS> मझ एक तो खरीदना ही होगा। <EOS>
-
Input sentence:  <SOS> i'll come back . <EOS>
Translation:  म म ह। <EOS> 
Truth value:  <SOS> म वापस आऊगा। <EOS>
-
Input sentence:  <SOS> i'll phone you . <EOS>
Translation:  हम मरा कर ह ह। <EOS> 
Truth value:  <SOS> म तमह फोन करगा। <EOS>
-
Input sentence:  <SOS> i'll stay home . <EOS>
Translation:  मझ वह बहत की  <EOS> 
Truth value:  <SOS> म घर पर ही रहगा। <EOS>
-
Input sentence:  <SOS> i'm an atheist . <EOS>
Translation:  कया ! <EOS> 
Truth value:  <SOS> म अनीशवरवादी ह। <EOS>
-
Input sentence:  <SOS> i'm an atheist . <EOS>
Translation:  वह म म म नही ह। <EOS> 
Truth value:  <SOS> म भगवान म यकीन न

-
Input sentence:  <SOS> she talks a lot . <EOS>
Translation:  हम बहत बहत बहत ह। <EOS> 
Truth value:  <SOS> वह बहत बक-बक करती ह। <EOS>
-
Input sentence:  <SOS> that isn't fair . <EOS>
Translation:  वह नही नही नही ह। ह। <EOS> 
Truth value:  <SOS> यह तो ठीक नही ह। <EOS>
-
Input sentence:  <SOS> that's my fault . <EOS>
Translation:  मझ मरा ह। ह। <EOS> 
Truth value:  <SOS> वह मरी गलती ह। <EOS>
-
Input sentence:  <SOS> the dog is mine . <EOS>
Translation:  वह ह। <EOS> 
Truth value:  <SOS> कतता मरा ह। <EOS>
-
Input sentence:  <SOS> the power's out . <EOS>
Translation:  म म म ह। <EOS> 
Truth value:  <SOS> बिजली गई हई ह। <EOS>
-
Input sentence:  <SOS> these are birds . <EOS>
Translation:  मरी म ह। <EOS> 
Truth value:  <SOS> य चिडिया ह। <EOS>
-
Input sentence:  <SOS> this is for you . <EOS>
Translation:  तम कल हो <EOS> 
Truth value:  <SOS> यह तमहार लिए ह। <EOS>
-
Input sentence:  <SOS> we lack nothing . <EOS>
Translation:  इस एक क क की ह ह। <EOS> 
Truth value:  <SOS> हम किसी भी चीज की कमी नही ह

-
Input sentence:  <SOS> didn't you go out ? <EOS>
Translation:  मझ म की को नही <EOS> 
Truth value:  <SOS> तम बाहर नही गए थ कया ? <EOS>
-
Input sentence:  <SOS> didn't you go out ? <EOS>
Translation:  म पास क बहत बहत ह <EOS> 
Truth value:  <SOS> त बाहर नही गया था कया ? <EOS>
-
Input sentence:  <SOS> do you believe me ? <EOS>
Translation:  वह उस एक स स ह ? <EOS> 
Truth value:  <SOS> कया तमह मझपर यकीन ह ? <EOS>
-
Input sentence:  <SOS> do you believe me ? <EOS>
Translation:  म तमह स ह ह ? <EOS> 
Truth value:  <SOS> कया तम मझपर यकीन करत हो ? <EOS>
-
Input sentence:  <SOS> do you believe me ? <EOS>
Translation:  मझ वह बहत ह ह ? <EOS> 
Truth value:  <SOS> कया तम मरा यकीन करत हो ? <EOS>
-
Input sentence:  <SOS> don't deceive him . <EOS>
Translation:  इस बहत मत ह। <EOS> 
Truth value:  <SOS> उस धोका मत दो। <EOS>
-
Input sentence:  <SOS> he began to shout . <EOS>
Translation:  कया ह <EOS> 
Truth value:  <SOS> वह चिललान लगा। <EOS>
-
Input sentence:  <SOS> he came to see me . <EOS>
Translation:  

-
Input sentence:  <SOS> he cried and cried . <EOS>
Translation:  म म म पर ह। <EOS> 
Truth value:  <SOS> वह रोया और और रोया। <EOS>
-
Input sentence:  <SOS> he is a simple man . <EOS>
Translation:  म म म ह ह। <EOS> 
Truth value:  <SOS> वह साधा आदमी ह। <EOS>
-
Input sentence:  <SOS> he is sure to come . <EOS>
Translation:  म ह ह ? <EOS> 
Truth value:  <SOS> वह जरर आएगा। <EOS>
-
Input sentence:  <SOS> he isn't my cousin . <EOS>
Translation:  म म म नही नही ह। <EOS> 
Truth value:  <SOS> वह मरा चचरा भाई नही ह। <EOS>
-
Input sentence:  <SOS> he isn't my cousin . <EOS>
Translation:  यह को को स ह। <EOS> 
Truth value:  <SOS> वह मरा ममरा भाई नही ह। <EOS>
-
Input sentence:  <SOS> he waited his turn . <EOS>
Translation:  म उस स नही कर ह। <EOS> 
Truth value:  <SOS> उसन अपनी बारी का इतजार किया। <EOS>
-
Input sentence:  <SOS> he works at a bank . <EOS>
Translation:  हम उस क ह। ह। <EOS> 
Truth value:  <SOS> वह बक म काम करता ह। <EOS>
-
Input sentence:  <SOS> he's a good person . <EOS>
Translation:  वह म

/home/akshat14714/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-2: 0.351674
BLEU-3: 0.534176
BLEU-4: 0.593021


In [0]:
def predict_sent(input_text):
    input_seq = []
    tmp = []
    text = preprocess_sentence(input_text)
    for word in text.split():
        embed = np.random.randn(EMBED_DIM)
        if word in eng_embedding:
            embed = eng_embedding[word]
        tmp.append(embed)
    input_seq.append(tmp)
    input_seq = pad_sequences(input_seq, max_input_seqlen, padding="post")
    states = encoder_model.predict(input_seq)
    target_seq = np.random.randn(EMBED_DIM)
    target_seq = [[target_seq]]
    target_text = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states)

        sampled_token_idx = np.argmax(output_tokens[0, -1, :])
        try:
            sampled_word = idx2targword[sampled_token_idx]
            target_text += sampled_word + " "
        except:
            target_text += " "

        if sampled_word == '\n' or len(target_text) >= max_target_seqlen:
            stop_condition = True

        if sampled_word in ger_embedding:
            target_seq = ger_embedding[sampled_word]
        else:
            target_seq = np.random.randn(EMBED_DIM)
        target_seq = [[target_seq]]

        states = [h, c]
    return target_text.strip()

In [0]:
predict_sent("Hello i am sourav.")

'म बहत ह ह। <EOS>'

In [0]:
!python OpenNMT-py/preprocess.py -train_src enghin/train.en -train_tgt enghin/train.hi -valid_src enghin/dev.en -valid_tgt enghin/dev.hi -save_data enghin/demo

In [0]:
!python OpenNMT-py/train.py -data enghin/demo -save_model enghin/demo-model